In [ ]:
import retrieval.cache as cache
import signals.ema as ema
import signals.simple_scalping as simple_scalping

import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

from backtesting import Backtest, Strategy
from tqdm import tqdm

### Overarching Constants

In [106]:
ticker = 'EURSEK=X'
start = '2023-01-01'
end = '2024-01-01'
interval = '1d'

columns_open = 'Open'
columns_high = 'High'
columns_low = 'Low'
columns_close = 'Close'
columns_volume = 'Volume'
columns_rsi = 'RSI'
columns_ema_slow = 'EMA Slow' # Need to change function calls below if change this one
columns_ema_fast = 'EMA Fast' # Need to change function calls below if change this one
columns_atr = 'ATR'
column_signal_ema = 'EMA Signal'
columns_simple_scalp = 'Simple Scalp'

refresh_cache=False
back_candle_length = 7

### Make sure we have some data to work with

In [107]:
if not cache.exist_sql_db(ticker, interval) or refresh_cache:
    # load data from yahoo finance and cache it in sql db
    cache.cache_ticker(ticker, interval, start, end)

# Load data from sql db
df = cache.load_ticker(ticker, interval, start, end)

df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-02,11.12888,11.19031,11.11657,11.12888,11.12888,0
1,2023-01-03,11.13104,11.16160,11.10648,11.13104,11.13104,0
2,2023-01-04,11.14480,11.18996,11.12794,11.14480,11.14480,0
3,2023-01-05,11.13811,11.26453,11.13273,11.13811,11.13811,0
4,2023-01-06,11.22406,11.29090,11.19627,11.22406,11.22406,0
...,...,...,...,...,...,...,...
255,2023-12-25,11.00437,11.07855,9.97513,11.00437,11.00437,0
256,2023-12-26,10.99726,11.09127,10.99287,10.99726,10.99726,0
257,2023-12-27,11.02066,11.09165,11.00573,11.02066,11.02066,0
258,2023-12-28,11.03764,11.05809,10.99899,11.03764,11.03764,0


### Add TA Columns

In [108]:
df[columns_ema_slow] = ta.ema(df[columns_close], length=50)
df[columns_ema_fast] = ta.ema(df[columns_close], length=30)
df[columns_rsi] = ta.rsi(df[columns_close], length=10)

# Average True Range (or Volatility) - help to define stop loss / take profit distance
df[columns_atr] = ta.atr(df[columns_high], df[columns_low], df[columns_close], length=back_candle_length)

# Bollinger bands
df=df.join(ta.bbands(df[columns_close], length=15, std=1.5))

#df


### EMA signal function

In [109]:
# Needed for progress_apply
tqdm.pandas()

df.reset_index(inplace=True)

# row.name is the index of the row -> 'Date' column
df[column_signal_ema] = df.progress_apply(
  lambda row: ema.two_above_or_below(df, row.name, back_candle_length), 
  axis=1
)

#df

100%|██████████| 260/260 [00:00<00:00, 3342.07it/s]


### Calculate the Simple Scalping Signal

In [110]:
df[columns_simple_scalp] = df.progress_apply(
  lambda row: simple_scalping.simple_signal(df, row.name, back_candle_length), axis=1
)

df[df['Simple Scalp'] != 0].head(200)

100%|██████████| 260/260 [00:00<00:00, 1706.63it/s]


,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA Slow,EMA Fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMA Signal,Simple Scalp
89,89,2023-05-05,11.25846,11.27820,11.19690,11.25846,11.25846,0,11.283985,11.306634,40.405851,0.085473,11.280159,11.325511,11.370862,0.800869,-0.239237,2,2
90,90,2023-05-08,11.20710,11.22460,11.17130,11.20710,11.20710,0,11.280970,11.300213,34.843274,0.085714,11.253800,11.316027,11.378254,1.099803,-0.375237,2,2
91,91,2023-05-09,11.18320,11.19347,11.16045,11.18320,11.18320,0,11.277135,11.292664,32.527918,0.080133,11.228061,11.307767,11.387474,1.409771,-0.281410,2,2
92,92,2023-05-10,11.17070,11.22804,11.13713,11.17070,11.17070,0,11.272962,11.284795,31.318538,0.081673,11.203792,11.298555,11.393319,1.677449,-0.174600,2,2
117,117,2023-06-14,11.52731,11.58763,11.47521,11.52731,11.52731,0,11.436122,11.501077,46.569645,0.097570,11.541637,11.609181,11.676726,1.163636,-0.106054,2,2
138,138,2023-07-13,11.54678,11.55928,11.40553,11.54678,11.54678,0,11.615770,11.693957,33.127658,0.158050,11.636711,11.770325,11.903939,2.270350,-0.336536,2,2
139,139,2023-07-14,11.45052,11.52012,11.42990,11.45052,11.45052,0,11.609290,11.678251,28.326539,0.152169,11.575469,11.753802,11.932136,3.034485,-0.350324,2,2
140,140,2023-07-17,11.49133,11.53575,11.44062,11.49133,11.49133,0,11.604664,11.666192,32.907038,0.144020,11.537540,11.740423,11.943305,3.456143,-0.113882,2,2
142,142,2023-07-19,11.46001,11.52471,11.43590,11.46001,11.46001,0,11.595480,11.643552,32.093704,0.130296,11.467092,11.708429,11.949766,4.122450,-0.014673,2,2
190,190,2023-09-25,11.84030,11.86208,11.68400,11.84030,11.84030,0,11.808724,11.859651,41.832970,0.110580,11.863951,11.901761,11.939570,0.635358,-0.312776,2,2


### Create Dots for Simple Scalping Signals

In [111]:
def render_short_long_signal_point(row: pd.Series, column_name:str) -> str:
  if row[column_name] == simple_scalping.LONG_SIGNAL:
    return row[columns_low]-1e-3
  elif row[column_name] == simple_scalping.SHORT_SIGNAL:
    return row[columns_high]+1e-3
  
  return np.nan

df['ss_point'] = df.apply(lambda row: render_short_long_signal_point(row, columns_simple_scalp), axis=1)

### Plot the Graph

In [112]:
partition = df

fig = go.Figure(data=[go.Candlestick(x=partition.index,
                open=partition[columns_open],
                high=partition[columns_high],
                low=partition[columns_low],
                close=partition[columns_close]),
                
                go.Scatter(x=partition.index, y=partition['BBL_15_1.5'], 
                           line=dict(color='blue', width=1), 
                           name='BBL'),

                go.Scatter(x=partition.index, y=partition['BBU_15_1.5'],
                            line=dict(color='blue', width=1),
                            name='BBU'),

                go.Scatter(x=partition.index, y=partition[columns_ema_slow],
                            line=dict(color='orange', width=1),
                            name='EMA Slow'),

                go.Scatter(x=partition.index, y=partition[columns_ema_fast],
                            line=dict(color='green', width=1),
                            name='EMA Fast'),

                go.Scatter(x=partition.index, y=partition[columns_rsi],
                            line=dict(color='red', width=1),
                            name='RSI'),
                ])

fig.add_trace(go.Scatter(x=partition.index, y=partition['ss_point'],
                            mode='markers', marker=dict(color='darkorange', size=5),
                            name=columns_simple_scalp))

### Backtesting

In [113]:
def signal():
    return df[columns_simple_scalp]

class MyStrategy(Strategy):
    # Stop loss coefficient
    sl_coef = 1.1
    tpsl_ratio = 1.5

    def init(self):
        super().init()

        self.simple_scalp_signal = self.I(signal)

    def next(self):
        super().next()

        # Stop-loss distance
        sl_atr = self.sl_coef * self.data.ATR[-1]
        tpsl_ratio = self.tpsl_ratio

        
        if self.simple_scalp_signal == simple_scalping.LONG_SIGNAL and len(self.trades) == 0:
            sl1 = self.data[columns_close][-1] - sl_atr
            tp1 = self.data[columns_close][-1] + sl_atr * tpsl_ratio
            self.buy(sl=sl1, tp=tp1)

        if self.simple_scalp_signal == simple_scalping.SHORT_SIGNAL and len(self.trades) == 0:
            sl1 = self.data[columns_close][-1] + sl_atr
            tp1 = self.data[columns_close][-1] - sl_atr * tpsl_ratio
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df, MyStrategy, cash=16000)

bt.run()

/var/folders/_w/4mgyjmws5qq_yf18_4s5kff80000gn/T/ipykernel_36798/3229494173.py:32: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                     259.0
Duration                                259.0
Exposure Time [%]                   25.384615
Equity Final [$]                 17015.853283
Equity Peak [$]                  17015.853283
Return [%]                           6.349083
Buy & Hold Return [%]               -0.886158
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.695816
Avg. Drawdown [%]                   -0.659404
Max. Drawdown Duration                   20.0
Avg. Drawdown Duration               5.444444
# Trades                                 10.0
Win Rate [%]                             50.0
Best Trade [%]                       3.118135
Worst Trade [%]                     -1.145301
Avg. Trade [%]                    

### Plot the Backtesting Results

In [ ]:
bt.plot()